In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os
import json
import seaborn as sns
from datetime import datetime
from scipy.stats import linregress

In [16]:
STOCKS = ['FROG', 'SOUN', 'CRWV']
REF_ORDER_SIZES = [100, 500, 1000, 5000]  # Added larger order size
MODELS = ['linear', 'sqrt', 'quadratic']
S = 10000  # Total shares to execute
OUTPUT_DIR = 'results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## DATA PROCESSING FUNCTIONS

In [17]:
def load_data(stock):
    """Load and preprocess order book data for a stock"""
    file_path = f'{stock}.csv'
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    
    try:
        # Load data with appropriate dtype optimization
        dtype = {f'bid_sz_{i:02d}': 'float32' for i in range(10)}
        dtype.update({f'ask_sz_{i:02d}': 'float32' for i in range(10)})
        dtype.update({f'bid_px_{i:02d}': 'float32' for i in range(10)})
        dtype.update({f'ask_px_{i:02d}': 'float32' for i in range(10)})
        
        df = pd.read_csv(file_path, 
                         parse_dates=['ts_event'], 
                         dtype=dtype,
                         low_memory=False)
        # Rename and filter columns
        df = df.rename(columns={'ts_event': 'timestamp'})
        df = df.drop_duplicates(subset=['timestamp'])
        df = df.set_index('timestamp')
        
        # Filter to trading hours (9:30 AM to 4:00 PM EST)
        df = df.between_time('09:30', '16:00')
        
        # Resample to 1-minute intervals (taking the first observation in each minute)
        df = df.resample('1T').first()
        
        print(f"Loaded {len(df)} minutes of data for {stock}")
        return df.dropna(subset=['bid_px_00'])
    
    except Exception as e:
        print(f"Error loading {stock} data: {str(e)}")
        return None

## SLIPPAGE CALCULATION

In [4]:
def calculate_slippage(row, order_size):
    """Calculate slippage for a given order size at a snapshot"""
    try:
        # Extract best bid/ask
        best_bid = row['bid_px_00']
        best_ask = row['ask_px_00']
        if pd.isna(best_bid) or pd.isna(best_ask):
            return np.nan
        
        mid_price = (best_bid + best_ask) / 2.0
        
        # Walk the order book for ask side (since we're buying)
        shares_remaining = order_size
        total_cost = 0.0
        last_price = best_ask
        
        for level in range(10):
            price_col = f'ask_px_{level:02d}'
            size_col = f'ask_sz_{level:02d}'
            
            if price_col not in row or size_col not in row:
                break
                
            price = row[price_col]
            volume = row[size_col]
            
            if pd.isna(price) or pd.isna(volume) or volume <= 0:
                continue
                
            if shares_remaining <= volume:
                total_cost += shares_remaining * price
                shares_remaining = 0
                break
            else:
                total_cost += volume * price
                shares_remaining -= volume
                last_price = price  # Track last price in case we exhaust levels
        
        # If we still have shares remaining, use last available price
        if shares_remaining > 0:
            total_cost += shares_remaining * last_price
        
        avg_price = total_cost / order_size
        return avg_price - mid_price
    
    except Exception as e:
        print(f"Error calculating slippage: {str(e)}")
        return np.nan

In [18]:
def calculate_available_liquidity(row, depth=10):
    """Calculate total available liquidity in the ask side"""
    try:
        total_volume = 0
        for level in range(depth):
            size_col = f'ask_sz_{level:02d}'
            if size_col in row:
                volume = row[size_col]
                if not pd.isna(volume):
                    total_volume += volume
        return total_volume
    except:
        return 0

## IMPACT MODELING FUNCTIONS

In [19]:
def fit_impact_models(sizes, slippages):
    """Fit different impact models to slippage data"""
    results = {}
    sizes = np.array(sizes)
    slippages = np.array(slippages)
    
    # Linear model: g(x) = beta * x
    try:
        beta = np.mean(slippages / sizes)
        results['linear'] = {'beta': beta, 'r_squared': None}
    except:
        results['linear'] = {'beta': np.nan, 'r_squared': np.nan}
    
    # Square root model: g(x) = k * sqrt(x)
    try:
        sqrt_sizes = np.sqrt(sizes)
        slope, intercept, r_value, p_value, std_err = linregress(sqrt_sizes, slippages)
        results['sqrt'] = {'k': slope, 'intercept': intercept, 'r_squared': r_value**2}
    except:
        results['sqrt'] = {'k': np.nan, 'intercept': np.nan, 'r_squared': np.nan}
    
    # Quadratic model: g(x) = a * x^2 + b * x
    try:
        X = np.column_stack((sizes**2, sizes))
        model = np.linalg.lstsq(X, slippages, rcond=None)[0]
        a, b = model
        results['quadratic'] = {'a': a, 'b': b, 'r_squared': None}
    except:
        results['quadratic'] = {'a': np.nan, 'b': np.nan, 'r_squared': np.nan}
    
    return results

## OPTIMIZATION FUNCTIONS

In [20]:
def optimize_execution(betas, volumes, total_shares, model='linear'):
    """Compute optimal execution schedule"""
    n_periods = len(betas)
    
    # Objective function
    def cost(x):
        if model == 'linear':
            return np.sum(betas * x)
        elif model == 'sqrt':
            return np.sum(betas * np.sqrt(x))
        elif model == 'quadratic':
            return np.sum(betas * (0.5 * x**2))  # Quadratic term only
        else:
            return np.sum(betas * x)
    
    # Constraints
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - total_shares},  # Total shares constraint
        {'type': 'ineq', 'fun': lambda x: volumes - x}  # Volume constraints
    )
    
    # Initial guess (proportional to available volume)
    x0 = total_shares * volumes / np.sum(volumes)
    
    # Solve optimization
    res = minimize(
        cost, 
        x0,
        method='SLSQP',
        bounds=[(0, v) for v in volumes],
        constraints=constraints,
        options={'maxiter': 1000, 'disp': False}
    )
    
    if not res.success:
        print(f"Optimization warning: {res.message}")
    
    return res.x

## VISUALIZATION OF DIFFERENT PARAMETERS

In [21]:
def plot_impact_curves(stock, sizes, slippages, models, timestamp):
    """Plot slippage vs order size with fitted models"""
    plt.figure(figsize=(10, 6))
    plt.scatter(sizes, slippages, s=100, label='Observed Slippage')
    
    x_vals = np.linspace(min(sizes), max(sizes)*1.2, 100)
    
    if 'linear' in models and not np.isnan(models['linear']['beta']):
        y_linear = models['linear']['beta'] * x_vals
        plt.plot(x_vals, y_linear, 'r-', label='Linear Model')
    
    if 'sqrt' in models and not np.isnan(models['sqrt']['k']):
        y_sqrt = models['sqrt']['k'] * np.sqrt(x_vals) + models['sqrt']['intercept']
        plt.plot(x_vals, y_sqrt, 'g--', label='Square Root Model')
    
    if 'quadratic' in models and not np.isnan(models['quadratic']['a']):
        y_quad = models['quadratic']['a'] * x_vals**2 + models['quadratic']['b'] * x_vals
        plt.plot(x_vals, y_quad, 'b:', label='Quadratic Model')
    
    plt.title(f'Temporary Impact Models - {stock}\n{timestamp}')
    plt.xlabel('Order Size (Shares)')
    plt.ylabel('Slippage (Dollars per Share)')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{OUTPUT_DIR}/{stock}_impact_models_{timestamp.strftime("%H%M")}.png')
    plt.close()

In [22]:
def plot_optimal_allocation(stock, allocation, betas, volumes, model):
    """Visualize the optimal execution schedule"""
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Plot allocation
    ax1.bar(range(len(allocation)), allocation, alpha=0.6, color='blue', label='Allocation')
    ax1.set_xlabel('Time Period (1-min intervals)')
    ax1.set_ylabel('Shares to Execute', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_ylim(0, max(allocation) * 1.2)
    
    # Plot beta values
    ax2 = ax1.twinx()
    ax2.plot(betas, 'r-', linewidth=2, label='Beta (Slippage Coefficient)')
    ax2.set_ylabel('Beta (Slippage per Share)', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    # Plot volume constraints
    ax1.plot(volumes, 'g--', alpha=0.7, label='Volume Constraint')
    
    plt.title(f'Optimal Execution Allocation - {stock} ({model} model)')
    fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.9))
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/{stock}_optimal_allocation_{model}.png')
    plt.close()

In [11]:
def plot_beta_comparison(stock_results):
    """Compare beta values across stocks and models"""
    plt.figure(figsize=(10, 6))
    stock_names = []
    linear_betas = []
    sqrt_betas = []  # Using k as comparable parameter
    
    for stock, results in stock_results.items():
        if 'avg_linear_beta' in results:
            stock_names.append(stock)
            linear_betas.append(results['avg_linear_beta'])
            sqrt_betas.append(results['avg_sqrt_k'] * 10)  # Scale for comparison
    
    x = np.arange(len(stock_names))
    width = 0.35
    
    plt.bar(x - width/2, linear_betas, width, label='Linear Beta')
    plt.bar(x + width/2, sqrt_betas, width, label='Sqrt Model K (scaled)')
    
    plt.xlabel('Stock')
    plt.ylabel('Slippage Coefficient')
    plt.title('Slippage Coefficient Comparison Across Stocks')
    plt.xticks(x, stock_names)
    plt.legend()
    plt.grid(axis='y')
    plt.savefig(f'{OUTPUT_DIR}/slippage_coefficient_comparison.png')
    plt.close()

## ANALYSIS OF STOCKS

In [23]:
def analyze_stock(stock, total_shares=S):
    """Full analysis for one stock"""
    print(f"\n{'='*40}")
    print(f"Analyzing {stock}")
    print(f"{'='*40}")
    
    # Load and prepare data
    df = load_data(stock)
    if df is None or df.empty:
        return None
    
    # Precompute slippage for various order sizes
    print("Calculating slippage for different order sizes...")
    for size in REF_ORDER_SIZES:
        df[f'slippage_{size}'] = df.apply(lambda row: calculate_slippage(row, size), axis=1)
    
    # Calculate available liquidity
    df['volume_constraint'] = df.apply(calculate_available_liquidity, axis=1)
    
    # Fit models for each time period
    print("Fitting impact models...")
    model_results = []
    
    for idx, row in df.iterrows():
        sizes = REF_ORDER_SIZES
        slippages = [row[f'slippage_{size}'] for size in sizes]
        
        # Skip if we have NaNs in slippages
        if any(np.isnan(slippages)):
            continue
        
        models = fit_impact_models(sizes, slippages)
        model_results.append({
            'timestamp': idx,
            'linear_beta': models['linear']['beta'],
            'sqrt_k': models['sqrt']['k'],
            'sqrt_r2': models['sqrt']['r_squared'],
            'quadratic_a': models['quadratic']['a'],
            'quadratic_b': models['quadratic']['b']
        })
        
        # Plot models for a few representative timestamps
        if len(model_results) < 5:  # First few timestamps
            plot_impact_curves(stock, sizes, slippages, models, idx)
    
    # Create model results dataframe
    model_df = pd.DataFrame(model_results).set_index('timestamp')
    
    # Calculate average model parameters
    avg_linear_beta = model_df['linear_beta'].mean()
    avg_sqrt_k = model_df['sqrt_k'].mean()
    avg_sqrt_r2 = model_df['sqrt_r2'].mean()
    
    print(f"Average linear beta: {avg_linear_beta:.6f}")
    print(f"Average sqrt model k: {avg_sqrt_k:.6f} (R²: {avg_sqrt_r2:.4f})")
    
    # Optimize execution for each model
    print("\nOptimizing execution schedules...")
    allocation_results = {}
    
    for model in MODELS:
        betas = model_df['linear_beta'].values  # Using linear beta as base
        volumes = df.reindex(model_df.index)['volume_constraint'].values
        
        # For non-linear models, we adjust beta to be comparable
        if model == 'sqrt':
            # Scale beta to make it comparable to linear model
            betas = avg_sqrt_k * np.ones_like(betas)
        elif model == 'quadratic':
            # Use a combination of linear and quadratic terms
            betas = 0.5 * model_df['quadratic_a'].values
        
        allocation = optimize_execution(betas, volumes, total_shares, model)
        allocation_results[model] = allocation
        
        # Save visualization
        plot_optimal_allocation(stock, allocation, betas, volumes, model)
        
        # Calculate total cost
        if model == 'linear':
            total_cost = np.sum(betas * allocation)
        elif model == 'sqrt':
            total_cost = np.sum(betas * np.sqrt(allocation))
        elif model == 'quadratic':
            total_cost = np.sum(model_df['quadratic_a'].values * allocation**2 + 
                              model_df['quadratic_b'].values * allocation)
        
        print(f"{model} model total cost: ${total_cost:.2f}")
    
    # Prepare results
    results = {
        'stock': stock,
        'num_observations': len(df),
        'avg_linear_beta': avg_linear_beta,
        'avg_sqrt_k': avg_sqrt_k,
        'avg_sqrt_r2': avg_sqrt_r2,
        'linear_allocation': allocation_results['linear'].tolist(),
        'sqrt_allocation': allocation_results['sqrt'].tolist(),
        'quadratic_allocation': allocation_results['quadratic'].tolist(),
        'analysis_date': datetime.now().isoformat()
    }
    
    # Save results
    with open(f'{OUTPUT_DIR}/{stock}_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"Analysis complete for {stock}")
    return results

In [24]:
if __name__ == "__main__":
    stock_results = {}
    
    for stock in STOCKS:
        results = analyze_stock(stock)
        if results:
            stock_results[stock] = results
    
    # Generate comparative analysis
    if stock_results:
        plot_beta_comparison(stock_results)
        
        # Create summary report
        report = {
            'analysis_date': datetime.now().isoformat(),
            'total_shares': S,
            'stocks_analyzed': list(stock_results.keys()),
            'results_summary': {}
        }
        
        for stock, res in stock_results.items():
            report['results_summary'][stock] = {
                'avg_linear_beta': res['avg_linear_beta'],
                'avg_sqrt_k': res['avg_sqrt_k'],
                'num_observations': res['num_observations']
            }
        
        with open(f'{OUTPUT_DIR}/summary_report.json', 'w') as f:
            json.dump(report, f, indent=2)
        
        print("\nAnalysis Summary:")
        print(f"Total shares to execute: {S}")
        print("Average slippage coefficients:")
        print("{:<10} {:<15} {:<15}".format('Stock', 'Linear Beta', 'Sqrt Model K'))
        for stock, res in stock_results.items():
            print("{:<10} {:<15.6f} {:<15.6f}".format(
                stock, res['avg_linear_beta'], res['avg_sqrt_k']))
        
        print(f"\nFull results and visualizations saved to: {OUTPUT_DIR}")
    else:
        print("No valid results generated")


Analyzing FROG
Loaded 150 minutes of data for FROG
Calculating slippage for different order sizes...
Fitting impact models...
Average linear beta: 0.000307
Average sqrt model k: 0.002953 (R²: 0.9590)

Optimizing execution schedules...
linear model total cost: $1.23
sqrt model total cost: $3.57
quadratic model total cost: $2.22
Analysis complete for FROG

Analyzing SOUN
Loaded 150 minutes of data for SOUN
Calculating slippage for different order sizes...
Fitting impact models...
Average linear beta: 0.000020
Average sqrt model k: 0.000164 (R²: 0.8910)

Optimizing execution schedules...
linear model total cost: $0.19
sqrt model total cost: $0.20
quadratic model total cost: $0.10
Analysis complete for SOUN

Analyzing CRWV
Loaded 150 minutes of data for CRWV
Calculating slippage for different order sizes...
Fitting impact models...
Average linear beta: 0.000322
Average sqrt model k: 0.006780 (R²: 0.9337)

Optimizing execution schedules...
linear model total cost: $0.72
sqrt model total co